In [1]:
#Import necessary modules
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

#Plot matplotlib plots directly within a notebook
%matplotlib inline

In [4]:
df = pd.read_csv('titanic_train.csv')
df2 = sns.load_dataset("titanic").copy()

print(df2.head())
df

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True  


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
X = df.drop('survived', axis =1)
y = df['survived']

KeyError: "['survived'] not found in axis"

In [ ]:
# Target & basic feature selection (avoid leakage columns like 'alive')
target = "survived"
features = ["pclass","sex","age","sibsp","parch","fare","embarked","alone"]

df = df[features + [target]].dropna(subset=[target])  # ensure target present
X = df[features]
y = df[target].astype(int)  # ensure binary 0/1

KeyError: "None of [Index(['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'alone',\n       'survived'],\n      dtype='object')] are in the [columns]"

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

num_cols = X.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X.select_dtypes(exclude=["number"]).columns.tolist()

numeric_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
])

categorical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocess = ColumnTransformer([
    ("num", numeric_pipe, num_cols),
    ("cat", categorical_pipe, cat_cols),
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

tree = Pipeline([
    ("prep", preprocess),
    ("clf", DecisionTreeClassifier(random_state=42))
])

param_grid = {
    "clf__max_depth": [3, 5, 7, 9, None],
    "clf__min_samples_split": [2, 5, 10, 20],
    "clf__min_samples_leaf": [1, 2, 5, 10],
}

grid = GridSearchCV(
    estimator=tree,
    param_grid=param_grid,
    scoring={"acc": "accuracy", "f1": "f1"},
    refit="f1",                  # pick best by F1 (more informative than accuracy here)
    cv=5,
    n_jobs=-1,
    verbose=0
)

grid.fit(X_train, y_train)
print("Best params:", grid.best_params_)
print("Best CV F1:", grid.best_score_)
best_tree = grid.best_estimator_

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = Pipeline([
    ("prep", preprocess),
    ("clf", RandomForestClassifier(
        n_estimators=300, max_depth=None, random_state=42, n_jobs=-1
    ))
])

rf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def eval_model(name, model, X_tr, y_tr, X_te, y_te):
    yhat_tr = model.predict(X_tr)
    yhat_te = model.predict(X_te)
    return {
        "Model": name,
        "Acc (train)": accuracy_score(y_tr, yhat_tr),
        "F1  (train)": f1_score(y_tr, yhat_tr),
        "Acc (test)":  accuracy_score(y_te, yhat_te),
        "F1  (test)":  f1_score(y_te, yhat_te),
    }

rows = []
rows.append(eval_model("DecisionTree (best)", best_tree, X_train, y_train, X_test, y_test))
rows.append(eval_model("RandomForest (300)", rf, X_train, y_train, X_test, y_test))
metrics_df = pd.DataFrame(rows).set_index("Model")
metrics_df.round(3)

In [ ]:
# Get feature names after preprocessing
prep = best_tree.named_steps["prep"]
ohe = prep.named_transformers_["cat"].named_steps["ohe"]
num_names = num_cols
cat_names = ohe.get_feature_names_out(cat_cols).tolist()
all_feature_names = num_names + cat_names

# Extract importances from the final tree step
clf = best_tree.named_steps["clf"]
importances = pd.Series(clf.feature_importances_, index=all_feature_names)
imp_sorted = importances.sort_values(ascending=False).head(15)

plt.figure(figsize=(8,5))
imp_sorted.plot(kind="barh")
plt.gca().invert_yaxis()
plt.title("Decision Tree Feature Importances (top 15)")
plt.xlabel("Importance")
plt.tight_layout()
plt.show()